I began by importing essential libraries for data handling, machine learning, and visualization. I loaded the training and testing data into pandas DataFrames to understand their structure. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
train_df1 = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df1.head()

,Id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,205743,01/09/2020 08:22,5.596350e+15,fraud_Berge LLC,gas_transport,72.18,Joseph,Moody,M,5901 Foster Pine,...,29.9912,-90.2479,68211,"Engineer, petroleum",23/04/1941,5223f3d810e0083482e47a5c3a1eba94,1378023768,30.424083,-90.080958,0.0
1,465349,12/12/2020 07:42,4.311370e+18,"fraud_Eichmann, Bogan and Rodriguez",gas_transport,36.66,Phillip,Delacruz,M,26678 Lisa Locks Apt. 904,...,40.7692,-103.0968,648,Community arts worker,21/09/1969,25f3a9b8613a9745c5802031bd3a2c62,1386834132,39.871664,-103.457961,0.0
2,17312,27/06/2020 13:03,2.264940e+15,fraud_Ankunding-Carroll,travel,1.74,Juan,Sherman,M,5939 Garcia Forges Suite 297,...,29.3641,-98.4924,1595797,Land,17/10/1995,2a6f3f7c697bc721a80078aefd68d857,1372338223,28.456118,-98.491336,0.0
3,234968,14/09/2020 04:12,2.224100e+15,"fraud_Schumm, Bauch and Ondricka",grocery_pos,140.99,Sheri,Thomas,F,019 Kimberly Light Apt. 039,...,30.7906,-81.5607,1293,Naval architect,11/09/1963,fcd03eb50b00fe682d8a47f3bec089a6,1379131971,31.292136,-82.002506,0.0
4,194255,28/08/2020 17:40,3.527540e+15,fraud_Gerlach Inc,shopping_net,1722.19,Teresa,Stein,F,2838 White Fields Apt. 473,...,32.9413,-81.1807,1725,Social researcher,05/12/1974,d3105e269285b69b334e99433d6a6585,1377711655,33.067620,-80.201090,0.0


I defined a function to calculate the age of customers at the time of their transactions, using their birthdate and the transaction date. 
Next, I calculated the average transaction amount by category and merged this information back into the training dataset. 
Finally, I created a new column to capture the difference between individual transaction amounts and their respective category averages. 

In [2]:
def calculate_age_at_trans(birthdate_str, trans_date_str):
    birthdate = datetime.strptime(birthdate_str, "%d/%m/%Y")
    trans_date = datetime.strptime(trans_date_str.split()[0], "%d/%m/%Y")  
    age = trans_date.year - birthdate.year - ((trans_date.month, trans_date.day) < (birthdate.month, birthdate.day))
    return age

train_df1['age_at_trans'] = train_df1.apply(lambda row: calculate_age_at_trans(row['dob'], row['trans_date_trans_time']), axis=1)
category_avg_amt = train_df1.groupby('category')['amt'].mean().reset_index(name='avg_amt')

train_df1 = train_df1.merge(category_avg_amt, on='category', how='left')

train_df1['amt_diff_from_category_avg'] = train_df1['amt'] - train_df1['avg_amt']

train_df1.head()


,Id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age_at_trans,avg_amt,amt_diff_from_category_avg
0,205743,01/09/2020 08:22,5.596350e+15,fraud_Berge LLC,gas_transport,72.18,Joseph,Moody,M,5901 Foster Pine,...,"Engineer, petroleum",23/04/1941,5223f3d810e0083482e47a5c3a1eba94,1378023768,30.424083,-90.080958,0.0,79,63.577001,8.602999
1,465349,12/12/2020 07:42,4.311370e+18,"fraud_Eichmann, Bogan and Rodriguez",gas_transport,36.66,Phillip,Delacruz,M,26678 Lisa Locks Apt. 904,...,Community arts worker,21/09/1969,25f3a9b8613a9745c5802031bd3a2c62,1386834132,39.871664,-103.457961,0.0,51,63.577001,-26.917001
2,17312,27/06/2020 13:03,2.264940e+15,fraud_Ankunding-Carroll,travel,1.74,Juan,Sherman,M,5939 Garcia Forges Suite 297,...,Land,17/10/1995,2a6f3f7c697bc721a80078aefd68d857,1372338223,28.456118,-98.491336,0.0,24,112.389683,-110.649683
3,234968,14/09/2020 04:12,2.224100e+15,"fraud_Schumm, Bauch and Ondricka",grocery_pos,140.99,Sheri,Thomas,F,019 Kimberly Light Apt. 039,...,Naval architect,11/09/1963,fcd03eb50b00fe682d8a47f3bec089a6,1379131971,31.292136,-82.002506,0.0,57,115.885327,25.104673
4,194255,28/08/2020 17:40,3.527540e+15,fraud_Gerlach Inc,shopping_net,1722.19,Teresa,Stein,F,2838 White Fields Apt. 473,...,Social researcher,05/12/1974,d3105e269285b69b334e99433d6a6585,1377711655,33.067620,-80.201090,0.0,45,83.481653,1638.708347


I calculated each job's average transaction amount, merged this data back into the main dataset, and created a column showing the difference from this average. This process adds valuable insights into how spending behavior varies by occupation, enriching my analysis.

In [3]:
job_avg_amt = train_df1.groupby('job')['amt'].mean().reset_index(name='job_avg_amt')

# 将平均交易金额的数据合并回原始数据集
train_df1 = train_df1.merge(job_avg_amt, on='job', how='left')

# 计算差值并将其作为新列添加到数据集中
train_df1['amt_diff_from_job_avg'] = train_df1['amt'] - train_df1['job_avg_amt']
train_df1.head()

,Id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,age_at_trans,avg_amt,amt_diff_from_category_avg,job_avg_amt,amt_diff_from_job_avg
0,205743,01/09/2020 08:22,5.596350e+15,fraud_Berge LLC,gas_transport,72.18,Joseph,Moody,M,5901 Foster Pine,...,5223f3d810e0083482e47a5c3a1eba94,1378023768,30.424083,-90.080958,0.0,79,63.577001,8.602999,57.975932,14.204068
1,465349,12/12/2020 07:42,4.311370e+18,"fraud_Eichmann, Bogan and Rodriguez",gas_transport,36.66,Phillip,Delacruz,M,26678 Lisa Locks Apt. 904,...,25f3a9b8613a9745c5802031bd3a2c62,1386834132,39.871664,-103.457961,0.0,51,63.577001,-26.917001,67.648849,-30.988849
2,17312,27/06/2020 13:03,2.264940e+15,fraud_Ankunding-Carroll,travel,1.74,Juan,Sherman,M,5939 Garcia Forges Suite 297,...,2a6f3f7c697bc721a80078aefd68d857,1372338223,28.456118,-98.491336,0.0,24,112.389683,-110.649683,40.964194,-39.224194
3,234968,14/09/2020 04:12,2.224100e+15,"fraud_Schumm, Bauch and Ondricka",grocery_pos,140.99,Sheri,Thomas,F,019 Kimberly Light Apt. 039,...,fcd03eb50b00fe682d8a47f3bec089a6,1379131971,31.292136,-82.002506,0.0,57,115.885327,25.104673,71.698189,69.291811
4,194255,28/08/2020 17:40,3.527540e+15,fraud_Gerlach Inc,shopping_net,1722.19,Teresa,Stein,F,2838 White Fields Apt. 473,...,d3105e269285b69b334e99433d6a6585,1377711655,33.067620,-80.201090,0.0,45,83.481653,1638.708347,60.606322,1661.583678


I encoded categorical variables like 'category', 'merchant', 'job', and 'gender' into numeric codes for model compatibility. I also segmented city populations into five categories for nuanced analysis. And I calculated each merchant's average transaction amount and merged it back.

In [4]:
train_df1['distance'] = np.sqrt((train_df1['merch_lat'] - train_df1['lat'])**2 + (train_df1['merch_long'] - train_df1['long'])**2)
train_df1['category_code'] = pd.factorize(train_df1['category'])[0]
train_df1['merchant_code'] = pd.factorize(train_df1['merchant'])[0]
train_df1['job_code'] = pd.factorize(train_df1['job'])[0]
train_df1['gender_code'] = pd.factorize(train_df1['gender'])[0]
train_df1['city_category'] = pd.qcut(train_df1['city_pop'], 5, labels=[1, 2, 3, 4, 5])
merchant_avg_amt = train_df1.groupby('merchant')['amt'].mean().reset_index(name='merchant_avg_amt')
train_df1 = train_df1.merge(merchant_avg_amt, on='merchant', how='left')

I further processed the data by categorizing transaction categories based on their suffixes, converting transaction dates to datetime objects for easier manipulation, and categorizing transactions by day type, checking for holidays, and extracting the hour of the transaction. I also introduced a method to classify transactions by season, added the transaction month, and encoded credit card numbers as numerical identifiers. Additionally, I created a unique user identifier combining credit card number, first name, and street address, aiming to enhance data granularity and facilitate more detailed behavioral analysis.

In [5]:
def categorize_by_suffix(category):
    if category.endswith('net'):
        return 1
    elif category.endswith('pos'):
        return 2
    else:
        return 0
train_df1['trans_date_trans_time'] = pd.to_datetime(train_df1['trans_date_trans_time'],format='%d/%m/%Y %H:%M')

train_df1['category_code1'] = train_df1['category'].apply(categorize_by_suffix)
train_df1['day_type'] =train_df1['trans_date_trans_time'].dt.weekday
import holidays
us_holidays = holidays.UnitedStates()
train_df1['is_holiday'] = train_df1['trans_date_trans_time'].apply(lambda x: x.date() in us_holidays)
train_df1['hour'] = train_df1['trans_date_trans_time'].dt.hour
def month_to_season(month):
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    elif month in [9, 10, 11]:
        return 4
train_df1['season'] = train_df1['trans_date_trans_time'].dt.month.apply(month_to_season)
train_df1['month'] = train_df1['trans_date_trans_time'].dt.month
train_df1['num_cc'] = pd.factorize(train_df1['cc_num'])[0]
train_df1['user_identifier'] = train_df1['cc_num'].astype(str) + '_' + train_df1['first'] + '_' + train_df1['street']

I calculated the average transaction amount per user and the difference from this average for each transaction, enhancing the dataset's depth for analyzing spending behavior. I then flagged transactions as occurring on weekends or during nighttime hours (defined as 9 PM to 6 AM) to explore temporal patterns. Additionally, I refined merchant identification by combining the merchant name with rounded latitude and longitude coordinates, creating a unique code, and then factorizing both this merchant code and the user identifier to numerical IDs, further preparing the dataset for sophisticated analytical models by ensuring all features are in a usable format.

In [6]:
train_df1['avg_amt_per_user'] = train_df1.groupby('user_identifier')['amt'].transform('mean')

# 步骤3: 计算消费与平均消费之差
train_df1['amt_diff_from_avg'] = train_df1['amt'] - train_df1['avg_amt_per_user']
train_df1['is_weekend'] = train_df1['trans_date_trans_time'].dt.dayofweek >= 5

# 计算是否夜间，定义为晚上9点到早上6点
train_df1['is_nighttime'] = train_df1['trans_date_trans_time'].dt.hour.apply(lambda x: x >= 21 or x < 6)
train_df1['merchant_code'] = (
    train_df1['merchant'] + '_' +
    train_df1['merch_lat'].round(4).astype(str) + '_' +
    train_df1['merch_long'].round(4).astype(str)
)
train_df1['merchant_id'] =pd.factorize(train_df1['merchant_code'])[0]
train_df1['customer_id']=pd.factorize(train_df1['user_identifier'])[0]

I applied K-Means clustering to segment cardholder and merchant locations into four distinct groups. This segmentation allows for the analysis of geographical patterns in transactions, potentially uncovering regional spending behaviors or fraud trends. By clustering both cardholders' and merchants' locations, I aim to enhance the model's ability to detect nuances in transaction data that are geographically influenced, thereby adding another layer of insight into consumer behavior and merchant distribution.（After utilizing the elbow method to identify that the optimal number of clusters is 4）

In [7]:
# Selecting cardholder's location data for clustering
X_cardholder = train_df1[['lat', 'long']]

# Applying K-Means Clustering
kmeans_cardholder = KMeans(n_clusters=4, random_state=42)
train_df1['cardholder_cluster'] = kmeans_cardholder.fit_predict(X_cardholder)

X_merchant = train_df1[['merch_lat', 'merch_long']]

kmeans_merchant = KMeans(n_clusters=4,random_state=42)
train_df1['merchant_cluster'] = kmeans_merchant.fit_predict(X_merchant)

C:\Users\10043\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\10043\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Following the feature engineering phase, I turned to model training, opting for XGBoost due to its superior performance among various classification models I considered. XGBoost is known for handling a wide range of data types, its efficiency, and its effectiveness in predictive modeling, especially in competitions and practical applications. I meticulously adjusted the feature set, selecting the combination that offered the best predictive power based on my dataset's unique characteristics. To optimize the model's hyperparameters, I employed grid search, a systematic approach that tests a range of predefined parameter values to find the combination that results in the most accurate predictions. This methodical tuning is crucial for enhancing model performance, ensuring that the final model is well-suited to uncovering insights and making accurate predictions based on my dataset.

In [15]:
df_train = train_df1.dropna(subset=['is_fraud'])

features = df_train.select_dtypes(include=[np.number]).columns.drop('is_fraud')
specified_columns = ['amt','zip','age_at_trans','lat','long','merch_lat','merchant_id','merch_long','city_pop',
                       'cardholder_cluster','merchant_cluster',
                    'is_nighttime','category_code1','job_code','job_avg_amt',
                     'customer_avg_amount_1_day', 'customer_avg_amount_7_day', 'customer_avg_amount_30_day',
                     'merchant_avg_amount_1_day','merchant_avg_amount_7_day', 'merchant_avg_amount_30_day',
                     'customer_transaction_count_1_day', 'customer_transaction_count_7_day', 'customer_transaction_count_30_day',
                     'merchant_transaction_count_1_day', 'merchant_transaction_count_7_day', 'merchant_transaction_count_30_day',
                     ,'amt_diff_from_category_avg','avg_amt','day_type','hour','num_cc','is_holiday','amt_diff_from_avg'] 

X = df_train[specified_columns]
y = df_train['is_fraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

param_grid = {
    'n_estimators': [626,630,625],
    'learning_rate': [ 0.29,0.30,0.31],  
    'max_depth': [4,5,6],  
    'colsample_bytree': [0.6,0.5,0.7],  
    'subsample': [1.0]  
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='f1',  
                           cv=3,  
                           verbose=1,  
                           n_jobs=-1)  

grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters found:  {'colsample_bytree': 0.6, 'learning_rate': 0.31, 'max_depth': 4, 'n_estimators': 630, 'subsample': 1.0}
Best accuracy found:  0.8574716801034526


In [20]:

model = xgb.XGBClassifier(use_label_encoder=False, 
                          eval_metric='logloss',
                          colsample_bytree=0.6,
                          learning_rate=0.31,
                          max_depth= 4,
                          n_estimators= 630,
                          subsample=1
                         ) 
model.fit(X_train, y_train)

y_pred_dt = model.predict(X_val)
print("XGBC:", f1_score(y_val, y_pred_dt))

XGBC: 0.8528974739970282


Given that the model's performance wasn't as promising as I had hoped, I decided to introduce more complex features to provide the model with a deeper understanding of the data.

To deepen the model's insight into transaction patterns, I implemented a rolling window analysis for both customers and merchants based on their transaction amounts and counts over specific periods: 1, 7, and 30 days. This method calculates the average transaction amount and the total number of transactions for each entity within these windows, providing a dynamic view of transactional behavior over time.

In [21]:
df_sorted = train_df1.copy()
df_sorted.sort_values(by='trans_date_trans_time', inplace=True)

for days in [1, 7, 30]:
    for entity in ['customer', 'merchant']:
        rolling_sum = df_sorted.groupby(f'{entity}_id')['amt'].rolling(window=days).sum().reset_index(0, drop=True)
        rolling_count = df_sorted.groupby(f'{entity}_id')['amt'].rolling(window=days).count().reset_index(0, drop=True)
        df_sorted[f'{entity}_avg_amount_{days}_day'] = rolling_sum / rolling_count
        df_sorted[f'{entity}_transaction_count_{days}_day'] = rolling_count
columns_to_fillna = ['customer_avg_amount_1_day', 'customer_avg_amount_7_day', 'customer_avg_amount_30_day',
                     'merchant_avg_amount_1_day','merchant_avg_amount_7_day', 'merchant_avg_amount_30_day',
                     'customer_transaction_count_1_day', 'customer_transaction_count_7_day', 'customer_transaction_count_30_day',
                     'merchant_transaction_count_1_day', 'merchant_transaction_count_7_day', 'merchant_transaction_count_30_day']
df_sorted[columns_to_fillna] = df_sorted[columns_to_fillna].fillna(0)
train_df1 = pd.merge(train_df1, df_sorted[columns_to_fillna + ['Id']], on='Id', how='left')
train_df1.head(10)

,Id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,customer_avg_amount_30_day,merchant_avg_amount_1_day,merchant_avg_amount_7_day,merchant_avg_amount_30_day,customer_transaction_count_1_day,customer_transaction_count_7_day,customer_transaction_count_30_day,merchant_transaction_count_1_day,merchant_transaction_count_7_day,merchant_transaction_count_30_day
0,205743,2020-09-01 08:22:00,5.596350e+15,fraud_Berge LLC,gas_transport,72.18,Joseph,Moody,M,5901 Foster Pine,...,55.237667,72.18,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
1,465349,2020-12-12 07:42:00,4.311370e+18,"fraud_Eichmann, Bogan and Rodriguez",gas_transport,36.66,Phillip,Delacruz,M,26678 Lisa Locks Apt. 904,...,80.953667,36.66,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
2,17312,2020-06-27 13:03:00,2.264940e+15,fraud_Ankunding-Carroll,travel,1.74,Juan,Sherman,M,5939 Garcia Forges Suite 297,...,32.056667,1.74,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
3,234968,2020-09-14 04:12:00,2.224100e+15,"fraud_Schumm, Bauch and Ondricka",grocery_pos,140.99,Sheri,Thomas,F,019 Kimberly Light Apt. 039,...,56.188333,140.99,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
4,194255,2020-08-28 17:40:00,3.527540e+15,fraud_Gerlach Inc,shopping_net,1722.19,Teresa,Stein,F,2838 White Fields Apt. 473,...,109.617000,1722.19,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
5,250314,2020-09-20 22:26:00,3.501510e+15,fraud_Gaylord-Powlowski,home,34.08,Frank,Anderson,M,0611 Stafford Valley Suite 504,...,70.396000,34.08,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
6,491809,2020-12-17 21:08:00,3.027300e+13,"fraud_Schaefer, Fay and Hilll",entertainment,50.20,Andrew,Patterson,M,06959 Stephen Branch Suite 246,...,44.555667,50.20,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
7,81552,2020-07-19 21:28:00,2.305340e+15,"fraud_Roberts, Ryan and Smith",personal_care,3.32,Jeffrey,Rice,M,21447 Powell Circle,...,53.855667,3.32,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
8,340015,2020-10-30 03:53:00,3.592930e+15,fraud_Reichel Inc,entertainment,5.47,Dylan,Garcia,M,923 Jordan Road,...,56.320000,5.47,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0
9,170293,2020-08-19 17:42:00,4.586810e+15,fraud_O'Hara-Wilderman,food_dining,1.23,Michelle,Gregory,F,6983 Carrillo Isle,...,76.624000,1.23,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0


In [23]:
df_train = train_df1.dropna(subset=['is_fraud'])

features = df_train.select_dtypes(include=[np.number]).columns.drop('is_fraud')
specified_columns = ['amt','zip','age_at_trans','lat','long','merch_lat','merchant_id','merch_long','city_pop',
                       'cardholder_cluster','merchant_cluster',
                    'is_nighttime','category_code1','job_code','job_avg_amt',
                     'customer_avg_amount_1_day', 'customer_avg_amount_7_day', 'customer_avg_amount_30_day',
                     'merchant_avg_amount_1_day','merchant_avg_amount_7_day', 'merchant_avg_amount_30_day',
                     'customer_transaction_count_1_day', 'customer_transaction_count_7_day', 'customer_transaction_count_30_day',
                     'merchant_transaction_count_1_day', 'merchant_transaction_count_7_day', 'merchant_transaction_count_30_day'
                     ,'amt_diff_from_category_avg','avg_amt','day_type','hour','num_cc','is_holiday','amt_diff_from_avg'] 

X = df_train[specified_columns]
y = df_train['is_fraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
model = xgb.XGBClassifier(use_label_encoder=False, 
                          eval_metric='logloss',
                          colsample_bytree=0.6,
                          learning_rate=0.26,
                          max_depth= 5,
                          n_estimators= 630,
                          subsample=1
                         ) 
model.fit(X_train, y_train)

y_pred_dt = model.predict(X_val)
print("XGBC:", f1_score(y_val, y_pred_dt))

XGBC: 0.9153515064562411


Upon analyzing the is_fraud column, I noticed a pattern where fraudulent activities for individual users were concentrated in recent, brief intervals. This insight led me to conceptualize a feature capturing the time elapsed since a user's last fraudulent transaction. I hypothesized that shorter intervals between such events might indicate a heightened risk of fraud. Implementing this feature involves tracking the time from each transaction to the preceding fraud incident for every user, thereby enriching the dataset with temporal dynamics that could significantly improve the model's ability to predict fraud.

In [30]:
def calculate_seconds_since_last_fraud_unix(group):
    group = group.sort_values('unix_time')
    fraud_timestamps = group['unix_time'][group['is_fraud'] == 1]
    seconds_since_last_fraud = group['unix_time'].apply(
        lambda x: (x - fraud_timestamps[fraud_timestamps < x]).min() if not fraud_timestamps[fraud_timestamps < x].empty else 365 * 24 * 3600)
    return seconds_since_last_fraud

train_df1['seconds_since_last_fraud_unix'] = train_df1.groupby('user_identifier').apply(calculate_seconds_since_last_fraud_unix).reset_index(level=0, drop=True)
train_df1.head(10)

,Id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,merchant_avg_amount_1_day,merchant_avg_amount_7_day,merchant_avg_amount_30_day,customer_transaction_count_1_day,customer_transaction_count_7_day,customer_transaction_count_30_day,merchant_transaction_count_1_day,merchant_transaction_count_7_day,merchant_transaction_count_30_day,seconds_since_last_fraud_unix
0,205743,2020-09-01 08:22:00,5.596350e+15,fraud_Berge LLC,gas_transport,72.18,Joseph,Moody,M,5901 Foster Pine,...,72.18,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
1,465349,2020-12-12 07:42:00,4.311370e+18,"fraud_Eichmann, Bogan and Rodriguez",gas_transport,36.66,Phillip,Delacruz,M,26678 Lisa Locks Apt. 904,...,36.66,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
2,17312,2020-06-27 13:03:00,2.264940e+15,fraud_Ankunding-Carroll,travel,1.74,Juan,Sherman,M,5939 Garcia Forges Suite 297,...,1.74,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
3,234968,2020-09-14 04:12:00,2.224100e+15,"fraud_Schumm, Bauch and Ondricka",grocery_pos,140.99,Sheri,Thomas,F,019 Kimberly Light Apt. 039,...,140.99,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
4,194255,2020-08-28 17:40:00,3.527540e+15,fraud_Gerlach Inc,shopping_net,1722.19,Teresa,Stein,F,2838 White Fields Apt. 473,...,1722.19,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
5,250314,2020-09-20 22:26:00,3.501510e+15,fraud_Gaylord-Powlowski,home,34.08,Frank,Anderson,M,0611 Stafford Valley Suite 504,...,34.08,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
6,491809,2020-12-17 21:08:00,3.027300e+13,"fraud_Schaefer, Fay and Hilll",entertainment,50.20,Andrew,Patterson,M,06959 Stephen Branch Suite 246,...,50.20,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
7,81552,2020-07-19 21:28:00,2.305340e+15,"fraud_Roberts, Ryan and Smith",personal_care,3.32,Jeffrey,Rice,M,21447 Powell Circle,...,3.32,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
8,340015,2020-10-30 03:53:00,3.592930e+15,fraud_Reichel Inc,entertainment,5.47,Dylan,Garcia,M,923 Jordan Road,...,5.47,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000
9,170293,2020-08-19 17:42:00,4.586810e+15,fraud_O'Hara-Wilderman,food_dining,1.23,Michelle,Gregory,F,6983 Carrillo Isle,...,1.23,0.0,0.0,1.0,7.0,30.0,1.0,0.0,0.0,31536000


In [31]:
df_train = train_df1.dropna(subset=['is_fraud'])

features = df_train.select_dtypes(include=[np.number]).columns.drop('is_fraud')
specified_columns = ['amt','zip','age_at_trans','lat','long','merch_lat','merchant_id','merch_long','city_pop',
                       'cardholder_cluster','merchant_cluster','seconds_since_last_fraud_unix',
                    'is_nighttime','category_code1','job_code','job_avg_amt',
                     'customer_avg_amount_1_day', 'customer_avg_amount_7_day', 'customer_avg_amount_30_day',
                     'merchant_avg_amount_1_day','merchant_avg_amount_7_day', 'merchant_avg_amount_30_day',
                     'customer_transaction_count_1_day', 'customer_transaction_count_7_day', 'customer_transaction_count_30_day',
                     'merchant_transaction_count_1_day', 'merchant_transaction_count_7_day', 'merchant_transaction_count_30_day'
                     ,'amt_diff_from_category_avg','avg_amt','day_type','hour','num_cc','is_holiday','amt_diff_from_avg'] 

X = df_train[specified_columns]
y = df_train['is_fraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
model = xgb.XGBClassifier(use_label_encoder=False, 
                          eval_metric='logloss',
                          colsample_bytree=0.7,
                          learning_rate=0.27,
                          max_depth= 5,
                          n_estimators= 625,
                          subsample=1
                         ) 
model.fit(X_train, y_train)

y_pred_dt = model.predict(X_val)
print("XGBC:", f1_score(y_val, y_pred_dt))

XGBC: 0.9612188365650969


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

param_grid = {
    'n_estimators': [625],
    'learning_rate': [ 0.29,0.28,0.27],  
    'max_depth': [5,6,7],  
    'colsample_bytree': [0.75,0.65,0.7],  
    'subsample': [1.0]  
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='f1',  
                           cv=3,  
                           verbose=1,  
                           n_jobs=-1)  

grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters found:  {'colsample_bytree': 0.7, 'learning_rate': 0.28, 'max_depth': 5, 'n_estimators': 625, 'subsample': 1.0}
Best accuracy found:  0.9595365679963369


From here I had 30/30 on the leaderboard.

In refining my model, I delved deeper into the training set and discovered a highly fraud-correlated feature: the ratio of fraudulent activities by the same cardholder on each date. Incorporating this insight as a new feature significantly enhanced the model's predictive accuracy by highlighting abnormal spending patterns within a day.

In [50]:
def calculate_daily_fraud_probability_with_id(df):
    df['temp_date'] = pd.to_datetime(df['trans_date_trans_time']).dt.date

    results = pd.DataFrame(index=df.index)
    
    for (user_id, date), group in df.groupby(['user_identifier', 'temp_date']):
        total_frauds = group['is_fraud'].sum()
        total_transactions = len(group)
        
        for idx in group.index:
            if group.at[idx, 'is_fraud'] == 1:
                prob = (total_frauds - 1) / total_transactions if total_transactions > 1 else 0
            else:
                prob = total_frauds / total_transactions
            results.at[idx, 'daily_fraud_probability'] = prob

    df['daily_fraud_probability'] = results['daily_fraud_probability']

    df.drop(columns=['temp_date'], inplace=True)

calculate_daily_fraud_probability_with_id(train_df1)



In [51]:
df_train = train_df1.dropna(subset=['is_fraud'])

features = df_train.select_dtypes(include=[np.number]).columns.drop('is_fraud')
specified_columns = ['amt','zip','age_at_trans','lat','long','merch_lat','merchant_id','merch_long','city_pop',
                       'cardholder_cluster','merchant_cluster','seconds_since_last_fraud_unix',
                    'is_nighttime','category_code1','job_code','job_avg_amt',
                     'customer_avg_amount_1_day', 'customer_avg_amount_7_day', 'customer_avg_amount_30_day',
                     'merchant_avg_amount_1_day','merchant_avg_amount_7_day', 'merchant_avg_amount_30_day',
                     'daily_fraud_probability',
                     'customer_transaction_count_1_day', 'customer_transaction_count_7_day', 'customer_transaction_count_30_day',
                     'merchant_transaction_count_1_day', 'merchant_transaction_count_7_day', 'merchant_transaction_count_30_day'
                     ,'amt_diff_from_category_avg','avg_amt','day_type','hour','num_cc','is_holiday','amt_diff_from_avg'] 

X = df_train[specified_columns]
y = df_train['is_fraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [52]:
model = xgb.XGBClassifier(use_label_encoder=False, 
                          eval_metric='logloss',
                          colsample_bytree=0.7,
                          learning_rate=0.27,
                          max_depth= 5,
                          n_estimators= 625,
                          subsample=1
                         ) 
model.fit(X_train, y_train)

y_pred_dt = model.predict(X_val)
print("XGBC:", f1_score(y_val, y_pred_dt))

XGBC: 0.9958734525447043


And got almost 100% in the local data.

In [49]:
test_features = train_df1.loc[train_df1['Id'].isin(test_df['Id'])]

X_test = test_features[specified_columns]

test_predictions = model.predict(X_test)

test_df['is_fraud_pred'] = test_predictions

test_df[['Id', 'is_fraud_pred']].to_csv('submission.csv', index=False)

In [ ]:
The final score on leaderboard was %98.125.